In [ ]:
# EV Routing pakcage
from ev_routing.cs_floyd_warshall import CSFloydWarshall
# Visualization packages
import plotly.graph_objects as go
import os


# Initializing Dijsktra
bl = {'lat': 52.510e0, 'lon': 13.383e0} # Bottom left corner coordinate
tr = {'lat': 52.520e0, 'lon': 13.391e0} # Top right corner coordinate

M = 30000

area = [bl['lat'], bl['lon'], tr['lat'], tr['lon']]
csfw = CSFloydWarshall(area, M, n_nodes=64, n_stations=8)

# Initializing bapbox
mapbox_access_token = os.environ['mapbox_access_token']

latitudes = [v['lat'] for v in csfw.v.values() if v['id'] in csfw.vid[:csfw.n_nodes] and v['id'] not in csfw.station_keys]
longitudes = [v['lon'] for v in csfw.v.values() if v['id'] in csfw.vid[:csfw.n_nodes] and v['id'] not in csfw.station_keys]

stations_latitudes = [v['lat'] for v in csfw.v.values() if v['id'] in csfw.vid[:csfw.n_nodes] and v['id'] in csfw.station_keys]
stations_longitudes = [v['lon'] for v in csfw.v.values() if v['id'] in csfw.vid[:csfw.n_nodes] and v['id'] in csfw.station_keys]

node_ids = csfw.vid[:csfw.n_nodes]
node_labels = [str(i) for i in range(len(csfw.v))]

In [ ]:
nodes_map = go.Figure()

nodes_map.add_trace( go.Scattermapbox(
    lat = latitudes,
    lon = longitudes,
    mode = 'markers',
    marker = go.scattermapbox.Marker(
        size = 12,
        color = 'rgb(242, 177, 172)',
        opacity=1.0
    ),
    text = node_labels,
    hoverinfo = 'text'
))

nodes_map.add_trace( go.Scattermapbox(
    lat = stations_latitudes,
    lon = stations_longitudes,
    mode = 'markers',
    marker = go.scattermapbox.Marker(
        size = 12,
        color = 'rgb(0, 0, 0)',
        opacity=1.0
    ),
    text = node_labels,
    hoverinfo = 'text'
))


nodes_map.update_layout(
    autosize = True,
    hovermode = 'closest',
    showlegend = False,
    mapbox = go.layout.Mapbox(
        accesstoken = mapbox_access_token,
        bearing = 0,
        center = go.layout.mapbox.Center(
            lat = csfw.map_center[0],
            lon = csfw.map_center[1]
        ),
        pitch = 0,
        zoom = 14,
        style = 'light'
    ),
)

nodes_map.update_layout(margin = {"r": 0, "t": 0, "l": 0, "b": 0})
nodes_map.show()

In [ ]:
csfw.run()